### Dataset

In this homework, we'll build a model for classifying various hair types. 
For this, we will use the Hair Type dataset that was obtained from 
[Kaggle](https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset) 
and slightly rebuilt.

You can download the target dataset for this homework from 
[here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip):

```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
unzip data.zip
```
The dataset is split into train and test dataset. Use train dataset to train the model and test dataset for validation. 

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

We will use PyTorch for that.

You can use Google Colab or your own computer for that.

### Data Preparation

The dataset contains around 1000 images of hairs in the separate folders 
for training and test sets. 

### Reproducibility

Reproducibility in deep learning is a multifaceted challenge that requires attention 
to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs.

Therefore, in this homework we suggest to set the random number seed generators by:

```python
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
```

Also, use PyTorch of version 2.8.0 (that's the one in Colab).

In [2]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Model

For this homework we will use Convolutional Neural Network (CNN). We'll use PyTorch.

You need to develop the model with following structure:

* The shape for input should be `(3, 200, 200)` (channels first format in PyTorch)
* Next, create a convolutional layer (`nn.Conv2d`):
    * Use 32 filters (output channels)
    * Kernel size should be `(3, 3)` (that's the size of the filter), padding = 0, stride = 1
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling (`nn.MaxPool2d`)
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using `flatten` or `view`
* Next, add a `nn.Linear` layer with 64 neurons and `'relu'` activation
* Finally, create the `nn.Linear` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use `torch.optim.SGD` with the following parameters:

* `torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)`

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class HairTypeCNN(nn.Module):
    def __init__(self):
        super(HairTypeCNN, self).__init__()

        # 1) CONVOLUTION LAYER: in_channels=3, out_channels=32, kernel_size=3

        # ---------------------------------------------------------
        #   "Add a convolution layer with 32 filters, kernel size 3,
        #    stride 1, padding 0"
        #
        # In TensorFlow:
        #   tf.keras.layers.Conv2D(32, 3, strides=1, padding="valid")
        #
        # In PyTorch:
        #   nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        #
        # Input shape:  (3, 200, 200)
        # Output shape: (32, 198, 198)
        # ---------------------------------------------------------

        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=32,
            kernel_size=3,
            stride=1,
            padding=0
        )

        # 2) MAX-POOL LAYER 2x2
        # In TensorFlow:
        #   tf.keras.layers.MaxPooling2D(pool_size=2)
        #
        # In PyTorch:
        #   nn.MaxPool2d(kernel_size, stride)
        #
        # Conv output: (32, 198, 198)
        # After pooling: (32, 99, 99)
        # ------------------------------

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # After conv + pool:
        # Input:  (3, 200, 200)
        # Conv:   (32, 198, 198)
        # Pool:   (32, 99, 99)

        # 3) COMPUTE FLATTEN SIZE
        # After Conv → Pool we know the tensor is:
        #   (batch, 32, 99, 99)
        # Flatten size = 32 * 99 * 99 = 313632
        self.flatten_size = 32 * 99 * 99

        # 4) FIRST FULLY CONNECTED LAYER (dense layer)
        #   "Add a fully-connected layer with 64 units"
        #
        # In TensorFlow:
        #   tf.keras.layers.Dense(64, activation="relu")
        #
        # PyTorch separates Linear + activation
        # ----------------------------------------
        self.fc1 = nn.Linear(self.flatten_size, 64)

        # 5) OUTPUT LAYER
        #   "Add the output layer with 1 neuron"
        #
        # Binary classification → output logits
        # (Note: use BCEWithLogitsLoss later)
        #
        # In TensorFlow:
        #   Dense(1, activation="sigmoid")
        #
        # In PyTorch:
        #   Linear() gives raw logits
        #   Sigmoid is applied inside the loss function
        self.fc2 = nn.Linear(64, 1)  # 1 output neuron for binary classification

    def forward(self, x):
        # INPUT SHAPE: (batch, 3, 200, 200)
        # x: (batch_size, 3, 200, 200)

        # 1) Convolution
        x = self.conv1(x)

        # Activation (PyTorch requires explicit activation)
        x = F.relu(x)

        # 2) Max Pooling
        x = self.pool(x)

        # 3) Flatten (TensorFlow does this automatically)
        x = x.view(x.size(0), -1)  # (batch_size, 313632)

        # 4) Dense layer + activation
        # Fully connected
        x = self.fc1(x)
        x = F.relu(x)

        # 5) Output layer
        x = self.fc2(x)  # logits, no sigmoid here

        return x


In [5]:
# instantiate the model and move it to the appropriate device
model = HairTypeCNN().to(device)


### Question 1

Which loss function you will use?

> `nn.BCEWithLogitsLoss()`

**ANSWER 👩🏽‍💻**

- I am doing binary classification with a single output neuron.
- My model returns raw logits (no sigmoid inside the model).
- `nn.BCEWithLogitsLoss()` is the correct loss for this setup because:
    - It expects one output per sample (logit), exactly like my model produces.
    - It internally applies a sigmoid + binary cross-entropy in a numerically stable way.
    - It is the PyTorch equivalent of using `Dense(1, activation="sigmoid")` + `BinaryCrossentropy` in TensorFlow.

The other options don't match the task:

- MSELoss treats the problem as regression → not ideal.
- CrossEntropyLoss requires two output classes, not a single logit.
- CosineEmbeddingLoss is for similarity learning, not classification.

So, `nn.BCEWithLogitsLoss()` is the correct and most stable loss for binary classification with a single-logit output.

### Question 2

What's the total number of parameters of the model? You can use `torchsummary` or count manually. 

In PyTorch, you can find the total number of parameters using:

```python
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")
```

> 20073473

In [7]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


### Generators and Training

For the next two questions, use the following transformation for both train and test sets.

* We don't need to do any additional pre-processing for the images.
* Use `batch_size=20`
* Use `shuffle=True` for both training, but `False` for test. 


In [10]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [24]:
validation_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [26]:
# datasets

train_dataset = datasets.ImageFolder(
    root='data/train',
    transform=train_transforms
)

validation_dataset = datasets.ImageFolder(
    root='data/test',
    transform=validation_transforms
)


In [27]:
# data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True
)

validation_loader = DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False
)


Now fit the model.

You can use this code:

In [28]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()


for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.5892, Acc: 0.6700, Val Loss: 0.6248, Val Acc: 0.6468
Epoch 2/10, Loss: 0.5431, Acc: 0.7050, Val Loss: 0.6500, Val Acc: 0.6219
Epoch 3/10, Loss: 0.5367, Acc: 0.7075, Val Loss: 0.6211, Val Acc: 0.6468
Epoch 4/10, Loss: 0.4570, Acc: 0.7812, Val Loss: 0.6164, Val Acc: 0.6517
Epoch 5/10, Loss: 0.3920, Acc: 0.8250, Val Loss: 0.6226, Val Acc: 0.6567
Epoch 6/10, Loss: 0.4134, Acc: 0.8125, Val Loss: 0.6186, Val Acc: 0.7065
Epoch 7/10, Loss: 0.3029, Acc: 0.8750, Val Loss: 0.7676, Val Acc: 0.6667
Epoch 8/10, Loss: 0.2699, Acc: 0.9000, Val Loss: 0.8655, Val Acc: 0.6368
Epoch 9/10, Loss: 0.2513, Acc: 0.8925, Val Loss: 0.6839, Val Acc: 0.7363
Epoch 10/10, Loss: 0.2078, Acc: 0.9137, Val Loss: 0.8861, Val Acc: 0.6766


## Question 3
What is the median of training accuracy for all the epochs for this model?

> 0.84

In [29]:
# median of training accuracy
median_acc = np.median(history['acc'])
print(f"Median training accuracy over all epochs: {median_acc:.4f}")

Median training accuracy over all epochs: 0.8187


## Question 4
What is the standard deviation of training loss for all the epochs for this model?

> 0.171


In [30]:
# std for training loss
std_loss = np.std(history['loss'])
print(f"Standard deviation of training loss over all epochs: {std_loss:.4f}")

Standard deviation of training loss over all epochs: 0.1280


### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

```python
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),

In [32]:
# Augmentation for training data

train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(
        size=200,
        scale=(0.9, 1.0),
        ratio=(0.9, 1.1),
    ),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])

# rebuild train dataset and loader with augmentation
train_dataset = datasets.ImageFolder(
    root="data/train",
    transform=train_transforms
)

validation_dataset = datasets.ImageFolder(
    root="data/test",
    transform=test_transforms
)

train_loader = DataLoader(
    train_dataset,
    batch_size=20,
    shuffle=True
)

validation_loader = DataLoader(
    test_dataset,
    batch_size=20,
    shuffle=False
)

### Question 5 

Let's train our model for 10 more epochs using the same code as previously.

> **Note:** make sure you don't re-create the model.
> we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

> 0.88


In [33]:
import numpy as np

num_epochs_aug = 10
test_losses = []

for epoch in range(num_epochs_aug):
    # ----- TRAIN -----
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:  # now with augmentation
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train

    # ----- TEST -----
    model.eval()
    test_running_loss = 0.0
    correct_test = 0
    total_test = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            test_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_test += labels.size(0)
            correct_test += (predicted == labels).sum().item()

    test_epoch_loss = test_running_loss / len(test_dataset)
    test_epoch_acc = correct_test / total_test
    test_losses.append(test_epoch_loss)

    print(
        f"Epoch {epoch+1}/{num_epochs_aug}, "
        f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, "
        f"Test Loss: {test_epoch_loss:.4f}, Test Acc: {test_epoch_acc:.4f}"
    )

# 👉 For Question 5:
mean_test_loss = np.mean(test_losses)
mean_test_loss


Epoch 1/10, Train Loss: 0.6816, Train Acc: 0.6212, Test Loss: 0.6085, Test Acc: 0.6816
Epoch 2/10, Train Loss: 0.5967, Train Acc: 0.6800, Test Loss: 0.6751, Test Acc: 0.6169
Epoch 3/10, Train Loss: 0.5923, Train Acc: 0.6550, Test Loss: 0.6860, Test Acc: 0.6567
Epoch 4/10, Train Loss: 0.5884, Train Acc: 0.6825, Test Loss: 0.6186, Test Acc: 0.6418
Epoch 5/10, Train Loss: 0.5860, Train Acc: 0.6813, Test Loss: 0.6601, Test Acc: 0.6468
Epoch 6/10, Train Loss: 0.5748, Train Acc: 0.7063, Test Loss: 0.6336, Test Acc: 0.6766
Epoch 7/10, Train Loss: 0.5607, Train Acc: 0.7087, Test Loss: 0.6214, Test Acc: 0.6766
Epoch 8/10, Train Loss: 0.5463, Train Acc: 0.7100, Test Loss: 0.6191, Test Acc: 0.6816
Epoch 9/10, Train Loss: 0.5161, Train Acc: 0.7412, Test Loss: 0.6259, Test Acc: 0.7015
Epoch 10/10, Train Loss: 0.5272, Train Acc: 0.7375, Test Loss: 0.6052, Test Acc: 0.6915


0.6353473142604923

### Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?
> 0.68


In [34]:
# Average test accuracy for last 5 epochs
avg_test_acc_last5 = np.mean(history['val_acc'][-5:])
print(f"Average test accuracy for last 5 epochs: {avg_test_acc_last5:.4f}")

Average test accuracy for last 5 epochs: 0.6846
